In [19]:
import time,serial,random
import numpy as np
  

'''
Receive acc.txt
Receive prediction.txt
Backprop alg
Update weight.txt
Update bias.txt
Correct acc.txt to be appropriate
Backward pass with initials acc.txt to angles of matrix.txt
Update matrix.txt
'''
def sigmoid(a):
    return 1.0/(1.0+np.exp(a))

def sigmoid_prime(a):
    return sigmoid(a)*(1.0-sigmoid(a))

def line(mtrx):
    return mtrx.reshape(mtrx.shape[0] * mtrx.shape[1],1)

if __name__ == "__main__":
    matrix = np.loadtxt('matrix.txt', 'int', delimiter='\t')
    acc = np.loadtxt('acc.txt', 'float', delimiter='\t')
    prediction = np.loadtxt('prediction.txt', 'float', delimiter='\t')
    layer = np.loadtxt('layer.txt', 'float', delimiter='\t')
    weight_1 = np.loadtxt('weight_1.txt', 'float', delimiter='|')
    weight_2 = np.loadtxt('weight_2.txt', 'float', delimiter='|')
    bias_1 = np.loadtxt('bias_1.txt', 'float', delimiter='\t')
    bias_2 = np.loadtxt('bias_2.txt', 'float', delimiter='\t')
    J = np.sum((acc-prediction)**2)
    matrix = matrix / 180.0
    acc = acc / 40.0 + 0.5
    delta_error = (prediction - acc) * prediction * (1 - prediction)
    delta_hidden = (weight_2.dot(line(delta_error))) * line(layer) * (1 - line(layer))
    delta_input = (weight_1.dot(line(delta_hidden))) * line(matrix) * (1 - line(matrix))